In [1]:
!pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup

In [3]:
import urllib
import  bs4

In [5]:
import re
import csv
import requests
from bs4 import BeautifulSoup

def save_to_csv(data, filename):
    with open(filename, 'w', encoding='utf-8', newline='') as csv_file:
        fieldnames = ["name", "description", "reduction", "website", "main_category", "subcategories"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for item in data:
            writer.writerow(item)

market_data_list_students = []
url_students = 'https://www.dell.com/fr-fr/lp/students'
response_students = requests.get(url_students)

if response_students.status_code == 200:
    soup_students = BeautifulSoup(response_students.text, 'html.parser')
    delltech_logo_link = soup_students.select_one('a.delltechLogoWrapper.dynamic-link')
    if delltech_logo_link:
        website = delltech_logo_link.get('href')
        print(f"Valeur de l'attribut href : {website}")
    else:
        print("Balise <a> avec la classe 'delltechLogoWrapper dynamic-link' non trouvée.")

    market_divs_students = soup_students.find_all('div', class_='rwp-contentlayout')
    first_name_encountered = False

    for i in market_divs_students:
        market_description_element = i.find('p', class_='rwp-contentlayout-item__description')
        reduction = None

        if market_description_element:
            if not first_name_encountered:
                market_name = market_description_element.text.split()[0]
                print(f"Name: {market_name}")
                first_name_encountered = True
            else:
                print("Additional names skipped")

            market_description = market_description_element.text.strip()
            print(f"Description: {market_description}")
            match = re.search(r'(\d+\s*%)', market_description)

            if match:
                reduction = match.group(1)
                print(f"Reduction: {reduction}")
            else:
                print("No percentage reduction found.")
        else:
            print("Description not found")

        market_data_list_students.append({
            "name": market_name,
            "description": market_description,
            "reduction": reduction,
            "website": website,
        })

    # Append the website attribute to each item in market_data_list_students
    for market_entry in market_data_list_students:
        market_entry["website"] = website

else:
    print(f"Failed to retrieve the page. Status code: {response_students.status_code}")

url_pc_deals = 'https://www.dell.com/fr-fr/shop/deals/top-pc-deals'
response_pc_deals = requests.get(url_pc_deals)

if response_pc_deals.status_code == 200:
    soup_pc_deals = BeautifulSoup(response_pc_deals.text, 'html.parser')
    category_items = soup_pc_deals.select('ul.sub-nav li.child-nav')

    categories_data = {}

    for category_item in category_items:
        main_category_name = category_item.select_one('button.mh-menuItem').text.strip()
        subcategory_buttons = category_item.select('li a.mh-menuItem')
        subcategories = [button.text.strip() for button in subcategory_buttons]
        categories_data[main_category_name] = subcategories

    combined_data_list = [
        {
            "name": market["name"],
            "description": market["description"],
            "reduction": market["reduction"],
            "website": market["website"],
            "main_category": main_category,
            "subcategories": ', '.join(subcategories)
        }
        for main_category, subcategories in categories_data.items()
        for market in market_data_list_students
    ]

    save_to_csv(combined_data_list, 'combined_dell_data.csv')

    print("Categories and Subcategories:")
    for main_category, subcategories in categories_data.items():
        print(f"Main Category: {main_category}")
        print(f"Subcategories: {', '.join(subcategories)}")
        print("---")
else:
    print(f"Failed to retrieve the page. Status code: {response_pc_deals.status_code}")


Valeur de l'attribut href : //www.dell.com/fr-fr
Name: DELL
Description: DELL TECHNOLOGIES POUR LES ÉTUDIANTSParé pour l’éducation.Préparé pour la vie.
No percentage reduction found.
Additional names skipped
Description: Remises exclusives pour les étudiantsObtenez jusqu’à 20 % de réduction sur les ordinateurs et les produits électroniques les mieux notés grâce à des bons de réduction exclusifs pour les étudiants. Si vous ne disposez pas encore de votre adresse e-mail académique, veuillez envoyer un e-mail à dell_advantage_student@dell.com pour demander votre bon de réduction.
Reduction: 20 %
Additional names skipped
Description: Solutions de recyclageLe recyclage de vos produits technologiques obsolètes constitue une solution responsable pour nous tous.
No percentage reduction found.
Additional names skipped
Description: Pour moins de déchetsNotre engagement en faveur d’une économie circulaire inclut l’exploitation de la technologie et des partenariats pour repenser, reconcevoir, réut

In [6]:
import pandas as pd
dell=pd.read_csv("/content/combined_dell_data.csv")
dell.head()
col = {'name': 'website_name', 'description': 'description','reduction':'Discount Percentage average','website':'website','main_category':'main_category','subcategories':'subcategories'
}
dell.rename(columns=col, inplace=True)
dell.head()
dell.to_csv("/content/dell.csv", index=False)